# Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:
* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

# Цель - найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.
## Алгоритм: 
* Скачайте файл с предложениями (sentences.txt).
* Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
* Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
* Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
* Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
* Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
* Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
* Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?**

$$ cos_{Alpha} = {u*v \over ||U||*||V||} $$

In [1]:
import re
import pandas as pd
from scipy import spatial

In [2]:
path = "database/text.txt"

In [3]:
with open(path, 'r') as txt:
    text = [i.lower() for i in txt.readlines()]

In [4]:
text

['in comparison to dogs, cats have not undergone major changes during the domestication process.\n',
 'as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.\n',
 'a common interactive use of cat for a single file is to output the content of a file to standard output.\n',
 'cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.\n',
 'in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.\n',
 'the domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.\n',
 'domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).\n',
 'however, if the output is piped or redirected, cat is unnecessary.\n',
 'cat with one named file is safer whe

In [5]:
sentences = [[el for el in re.split('[^a-z]', i) if el != ''] for i in text]

In [6]:
len(sentences)

22

In [7]:
all_words = set([el for k in text for el in re.split('[^a-z]', k)  if el != ''])

In [8]:
len(all_words)

254

In [9]:
data = pd.DataFrame({col : [] for col in all_words}, dtype=int  )

In [10]:
data

,by,felis,flow,useful,were,major,default,the,redirection,without,...,redirected,in,versions,vermin,bytes,lion,catenates,tiger,more,store


In [11]:
def frequency(sentences, all_words):
    return {el : sentences.count(el) for el in all_words}

In [12]:
def add_line(data_frame, freq, sentences, all_words):
    for i in sentences:
        data_frame = data_frame.append(freq(i, all_words),  ignore_index=True)
    return data_frame

In [13]:
data = add_line(data, frequency,sentences, all_words)

In [14]:
data.head()

,by,felis,flow,useful,were,major,default,the,redirection,without,...,redirected,in,versions,vermin,bytes,lion,catenates,tiger,more,store
0,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,0


In [15]:
data.shape

(22, 254)

In [16]:
one = list(data.iloc[0])

In [17]:
res = {i: spatial.distance.cosine(one, data.iloc[i]) for i in range(len(data))}

In [18]:
res

{0: 0.0,
 1: 0.9527544408738466,
 2: 0.8644738145642124,
 3: 0.8951715163278082,
 4: 0.7770887149698589,
 5: 0.9402385695332803,
 6: 0.7327387580875756,
 7: 0.9258750683338899,
 8: 0.8842724875284311,
 9: 0.9055088817476932,
 10: 0.8328165362273942,
 11: 0.8804771390665607,
 12: 0.8396432548525454,
 13: 0.8703592552895671,
 14: 0.8740118423302576,
 15: 0.9442721787424647,
 16: 0.8406361854220809,
 17: 0.956644501523794,
 18: 0.9442721787424647,
 19: 0.8885443574849294,
 20: 0.8427572744917122,
 21: 0.8250364469440588}

In [19]:
list(res.values()).index(min(list(res.values())[1:]))

6